# Consolidated Tickets Visuals

The tables and visualizations presented in this notebook aim to describe the underlying distribution of tickets from Remedy Reporting. The data used includes tickets from July 1, 2021 to August 30, 2021.

In [1]:
!pip3 install plotly
!pip3 install datetime
!pip3 install isoweek

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import plotly.express as px
from datetime import datetime, date
from isoweek import Week

In [3]:
tickets = pd.read_csv("20210830_Consolidated_Ticket.csv")

In [4]:
tickets

,Needs Attention,Smart IT Link,Status,Create Date,Customer First Name,Customer Last Name,Customer Organization,Customer Department,Support Company,Support Organization,Support Group Name,Service,Summary
0,No,INC000000883544,Closed,7/1/21 9:17,Sehrangez,Blackburn,The George Washington University,TBD,Division of IT,Technology Service Delivery,ITSC,NetID Self Service,Customer is requesting a NetID password reset
1,No,INC000000883546,Closed,7/1/21 9:33,Jenny,Dave,AA-SCHOOL OF MEDICINE AND HEALTH SCIENCES,MEDICINE-GME-INSTRUCTION,Division of IT,Technology Service Delivery,ITSC,Email and Calendaring,Email
2,No,INC000000883550,Closed,7/1/21 10:50,Guest,Account,NaN,NaN,Division of IT,Technology Service Delivery,ITSC,NaN,User requesting pw reset for MFA email account
3,No,INC000000883551,Closed,7/1/21 10:44,Rebecca (Rebecca),Wilkinson,AA-COLUMBIAN COLLEGE OF ARTS AND SCIENCES,ADMINISTRATION-ART THERAPY,Division of IT,Technology Service Delivery,ITSC,NaN,Faculty unable to access email accounts.
4,No,INC000000883553,Closed,7/1/21 10:58,Tiffany,Oie,The George Washington University,TBD,Division of IT,Technology Service Delivery,ITSC,GWEB Information Service (Banner Self Service),Customer requesting GWeb PIN reset or securit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11321,NaN,REQ000000290754,Pending,8/30/21 9:07,Arun Kumar,Doppalapudi Naga,EVPT-VP FINANCE AND TREASURER,PROCURE TO PAY,Division of IT,Technology Service Delivery,ITSC,NaN,Financial Systems Help Request
11322,NaN,REQ000000290839,Planning,8/30/21 7:49,Jorge,Walter,AA-SCHOOL OF BUSINESS,ADMIN-STRATEGIC MGT & PUBLIC POLICY,Division of IT,Technology Service Delivery,ITSC,NaN,Desktop phone for Staff or Faculty - request n...
11323,NaN,REQ000000290845,Planning,8/30/21 8:39,John,Gillum,AA-SCHOOL OF MEDICINE AND HEALTH SCIENCES,PHYSICIANS ASSITANT PROGRAM,Division of IT,Technology Service Delivery,ITSC,NaN,ITSC Hardware Support Check-In
11324,NaN,REQ000000290863,Completed,8/30/21 9:47,Michael,Hicks,EVPT-DIVISION OF INFORMATION TECHNOLOGY,SERVICE MANAGEMENT,Division of IT,Technology Service Delivery,ITSC,NaN,Call Centers - New or change request


In [7]:
tickets.groupby('Support Group Name', sort = True).size().sort_values(ascending=False).reset_index(name = "Count")


,Support Group Name,Count
0,ITSC,8151
1,LAW,1041
2,GWSPH / LAI / SEAS,564
3,SMHS,453
4,CCAS / ESIA,418
5,SON / VSTC / CPS VA Centers,381
6,CPS Admin / GSEHD / GWSB,318


In [9]:
tickets.groupby('Status').size().sort_values(ascending=False).reset_index(name = "Count")

,Status,Count
0,Closed,6175
1,Resolved,1942
2,Assigned,1342
3,Cancelled,675
4,Completed,400
5,Pending,350
6,In Progress,292
7,Planning,142
8,Rejected,5
9,Draft,2


In [10]:
tickets["Create Date"] = pd.to_datetime(tickets["Create Date"]).dt.date.astype('datetime64')

In [11]:
tickets['Week'] = tickets["Create Date"].dt.isocalendar()['week']

In [13]:
week_group_count = tickets.groupby(['Week', 'Support Group Name']).size().reset_index(name = "Count")
week_group_count


,Week,Support Group Name,Count
0,26,CCAS / ESIA,7
1,26,CPS Admin / GSEHD / GWSB,4
2,26,GWSPH / LAI / SEAS,14
3,26,ITSC,245
4,26,LAW,28
...,...,...,...
65,35,GWSPH / LAI / SEAS,14
66,35,ITSC,119
67,35,LAW,16
68,35,SMHS,7


In [14]:
week_group_count['Week Date'] = [Week(2021, week - 1).sunday() for week in week_group_count['Week']]

In [21]:
fig = px.line(week_group_count, x= 'Week Date', y= 'Count', color= 'Support Group Name', 
              title = "Tickets over Time by Support Group")
fig.show()

In [17]:
week_status_count = tickets.groupby(['Week', 'Status']).size().rename('Count').reset_index(name= 'Count')
week_status_count

,Week,Status,Count
0,26,Assigned,3
1,26,Cancelled,29
2,26,Closed,285
3,26,Completed,1
4,26,In Progress,7
...,...,...,...
81,35,Completed,1
82,35,In Progress,7
83,35,Pending,12
84,35,Planning,4


In [18]:
week_status_count['Week Date'] = [Week(2021, week - 1).sunday() for week in week_status_count['Week']]

In [22]:
fig2 = px.line(week_status_count, x= 'Week Date', y= 'Count', color= 'Status', hover_name='Status', 
               title= 'Tickets over time by Status')
fig2.show()

In [23]:
fig3 = px.bar(week_status_count, x= 'Count', y= 'Week Date', orientation = 'h', color= 'Status', hover_name='Status',
               title = "Status Breakdown by Week")
fig3.show()

In [24]:
fig4 = px.bar(week_group_count, x= 'Count', y= 'Week Date', orientation = 'h', color= 'Support Group Name', hover_name='Support Group Name',
               title = "Support Group Breakdown by Week")

fig4.show()

In [25]:
breakdown= tickets.groupby(['Week', 'Support Group Name', 'Status']).size().rename('Count').reset_index(name= 'Count')
breakdown

,Week,Support Group Name,Status,Count
0,26,CCAS / ESIA,Assigned,2
1,26,CCAS / ESIA,Cancelled,1
2,26,CCAS / ESIA,Closed,4
3,26,CPS Admin / GSEHD / GWSB,Assigned,1
4,26,CPS Admin / GSEHD / GWSB,Cancelled,1
...,...,...,...,...
352,35,SMHS,Assigned,6
353,35,SMHS,Resolved,1
354,35,SON / VSTC / CPS VA Centers,Assigned,2
355,35,SON / VSTC / CPS VA Centers,Pending,1


In [26]:
breakdown['Week Date'] = [Week(2021, week - 1).sunday() for week in breakdown['Week']]


Week                   int64
Support Group Name    object
Status                object
Count                  int64
Week Date             object
dtype: object

In [27]:
for i in range(len(breakdown)):
    breakdown['Week Date'][i] = str(breakdown['Week Date'][i])


<ipython-input-27-c01020e1213a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
fig5 = px.bar(breakdown, x= 'Support Group Name', y= 'Count', color= 'Status', facet_col = "Week Date", facet_col_wrap= 5,
              barmode= 'stack', title = "Consolidated Tickets Overview")

fig5.show()

fig5.write_html('fig5.html', auto_open = True)